# Session 5- Batch Analysis
*Goal - learn how to use the QuPath-created workflow to apply your analysis pipeline to multiple images*

## 1. Generating the Raw Workflow  Script
1. To repeat the analysis we've performed so far on a new image, we need to reproduce a few major steps:
  1. Segment the tissue with the "Tissue" pixel classifier
  2. Detect cells
  3. Segment the cancer with the "Tumor" pixel classifier

  This is different from the order we originally performed the steps in. Creating the Tumor object *after* cell detection will prevent it from being deleted. 
<br>
  
2. Start with LungImg2 open (the version with InstanSeg cell detections)
3. Open the Workflow tab. It will have a long list of commands that you have tried. 

  <img src="Images/WorkflowTab.PNG"> <br>

  **Don't be scared!** It looks overwhelming, but it just a list of buttons you've pressed. <br>

4. At the bottom of the screen, click <kbd>Create workflow</kbd> 

5. You'll get a window with a list of commands you've run. We're going to simplify that list by deleting anything unnecessary for analyzing the other images in the project. Therefore, delete (`highlight > right click > remove selected items`):
  - All of the duplicates of "Set image type" , leaving only 1 copy 
  - All of the "Cell detection" lines that ran the built-in cell detection, because we are going to use InstanSeg 
  - All of the "Delete selected objects" or "Clear detections" commands, because they are only useful during optimization. 
  - All but the last "Run InstanSeg model" command
    <img src="Images/DeleteWorkflowCommands.gif">
  
6. This will leave you with a much shorter list of steps, though it still needs further editing.  Click `Create script` at the bottom.
7. Expand the Script Editor window to make it easier to read. It will look something like this (your exact script will be different).

<img src="Images/ScriptEx1.PNG"> <br>

## 2. Editing the Script to do what we want
From here, we need to remove some extraneous lines and rearrange others. Step-by-step, here is what we need the script to do:

1. Set Image Type (`setImageType('FLUORESCENCE');`) - this declares that this is a fluorescence image with multiple channels. 
2. Segment the whole tissue. This will be one of the lines that starts `createAnnotationsFromPixelClassifier`. You will have a few of those in your starting script- find the one that uses the Tissue classifier and has the correct size parameters. Copy-paste it to be the second line in the script. 

    <img src="Images/ScriptLine2.gif">
    <br><br>
    
3. We need to select the annotation we just made before detecting cells. We can do that with the command `selectObjectsByClassification("Region*");` which selects all objects with the Region* class.  Copy-paste that to be the third line in the script.  

4. Then, we segment the cells in the whole tissue. The InstanSeg function to do this is multiple lines, starting with `qupath.ext.instanseg.core.InstanSeg.builder()` and ending with an indented line `    .detectObjects()`. Copy-paste that ENTIRE section to be the next part of the the code. 

  <img src="Images/InstanSegScriptLIne2.gif">
    
    

4. Last, we create the tumor annotation. This was also done with the function `createAnnotationsFromPixelClassifier`, but this time look for the one that says "TumorPixelClassifier" (or whatever you named your classifier).  Copy-paste that to be line 4.  Your parameters may look different than mine.

  <img src="Images/Script_TumorLine.PNG"> <br>

 
6. Delete all other lines after the `createAnnotationsFromPixelClassifier("TumorPixelClassifier", 500.0, 500.0)`
<br>

7. Save the script! In the Script Editor window, `File > Save As`. This will generate a Scripts folder inside your project folder. Give it a meaningful name. 

    <img src="Images/ScriptingSaveAs.PNG">

8. My finished script is pasted below. If you need to, you can copy-paste it. However *You must change the path in line 5* to the location where QuPath downloaded the model on your computer. I recommend getting this from your workflow.


In [ ]:
setImageType('FLUORESCENCE');
createAnnotationsFromPixelClassifier("Tissue", 1000.0, 1000.0, "INCLUDE_IGNORED")
selectObjectsByClassification("Region*");
qupath.ext.instanseg.core.InstanSeg.builder()
    .modelPath("C:/Users/smcardle/Documents/InstanSeg Models/fluorescence_nuclei_and_cells") 
    .device("cpu")
    .inputChannels([ColorTransforms.createChannelExtractor("Hoechst"), ColorTransforms.createChannelExtractor("CD11c"), ColorTransforms.createChannelExtractor("CD68"), ColorTransforms.createChannelExtractor("CD163"), ColorTransforms.createChannelExtractor("CD20"), ColorTransforms.createChannelExtractor("CD4"), ColorTransforms.createChannelExtractor("CD8a"), ColorTransforms.createChannelExtractor("CD45RO"), ColorTransforms.createChannelExtractor("PD1"), ColorTransforms.createChannelExtractor("CD45"), ColorTransforms.createChannelExtractor("S100a")])
    .outputChannels()
    .tileDims(512)
    .interTilePadding(16)
    .nThreads(4)
    .makeMeasurements(false)
    .randomColors(false)
    .build()
    .detectObjects()
createAnnotationsFromPixelClassifier("TumorPixelClassifier", 500.0, 500.0)


## 3. Run the script
We've gotten through the hardest part! Now, test the script.
1. Delete all objects in your current image. `Objects > Delete > Delete all objects`

2. In the script editor, `Run > Run` or press <kbd> Ctrl </kbd> + <kbd> R </kbd> 

  <img src="Images/RunRun.PNG">
  
   - If it worked, 2 annotations and thousands of cells appeared. If you see this HOORAY! 
   - If not, there is an error somewhere. It will tell you about the error at the bottom of the Script Editor in red. If you are in the room at the LJI Workshop, raise your hand. Otherwise, ask a question at forum.image.sc and include that error message. 

3. Now it's time to run on the other images. In the script editor <kbd> Run > Run for project </kbd>

4. Select LungImg1.ome.tiff and LungImg3.ome.tiff (or whichever 2 images you have not yet processed). You can highlight them both at the same time by holdring <kbd> Ctrl </kbd>. Then, click the <kbd> < </kbd> to choose those images.
      <img src="Images/RunForProject.PNG">
  
5. Hooray! You just ran your first batch script!
